In [269]:
import cv2
from skimage.exposure import cumulative_distribution
import numpy as np

box_points = []
button_down = False


def click_and_crop(event, x, y, flags, param):
    global box_points, button_down
    if (button_down == False) and (event == cv2.EVENT_LBUTTONDOWN):
        button_down = True
        box_points = [(x, y)]
    elif (button_down == True) and (event == cv2.EVENT_MOUSEMOVE):
        image_copy = param.copy()
        point = (x, y)
        cv2.rectangle(image_copy, box_points[0], point, (0, 255, 0), 2)
        cv2.imshow("Template Cropper - Press C to Crop", image_copy)
    elif event == cv2.EVENT_LBUTTONUP:
        button_down = False
        box_points.append((x, y))
        cv2.rectangle(param, box_points[0], box_points[1], (0, 255, 0), 2)
        cv2.imshow("Template Cropper - Press C to Crop", param)


# GUI template cropping tool
def template_crop(image):
    clone = image.copy()
    cv2.namedWindow("Template Cropper - Press C to Crop")
    param = image
    cv2.setMouseCallback("Template Cropper - Press C to Crop", click_and_crop, param)
    while True:
        cv2.imshow("Template Cropper - Press C to Crop", image)
        key = cv2.waitKey(1)
        if key == ord("c"):
            cv2.destroyAllWindows()
            break
    if len(box_points) == 2:
        cropped_region = clone[box_points[0][1]:box_points[1][1], box_points[0][0]:box_points[1][0]]
    return cropped_region


def read_raw_picture(path):
    import rawpy
    import imageio
    with  rawpy.imread(path) as raw:
        rgb = raw.postprocess()
    return cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)


def read_picture(img_path, templ_path):
    if img_path.split(".")[1] != 'dng':
        img = cv2.imread(img_path)
        templ = cv2.imread(templ_path)
        dng_ind = False
    else:
        img = read_raw_picture(img_path)
        templ = cv2.imread(templ_path)
        dng_ind = True
    img_res = cv2.resize(img, None, fx=0.15, fy=0.15)
    templ = cv2.resize(templ, None, fx=0.15, fy=0.15)
    return img_res, templ, dng_ind


def get_max_scaling(image, template):
    hi, wi, _ = image.shape
    ht, wt, _ = template.shape
    h_ratio = hi / ht
    w_ratio = wi / wt
    if h_ratio > w_ratio:
        percent = w_ratio
    else:
        percent = h_ratio
    print(f"Template sme da se povećava do {percent * 100}% svoje originalne veličine.")
    return percent, ht, wt


def split_interval(start, end, num_intervals):
    interval_points = np.linspace(start, end, num_intervals + 1)
    intervals = [(interval_points[i], interval_points[i + 1]) for i in range(num_intervals)]
    return intervals


def get_best_local_match(img_res, templ, lower_scale, upper_scale, percent, template_method):
    final_points = []
    hi, wi, _ = img_res.shape
    ht, wt, _ = templ.shape
    img_res = cv2.bilateralFilter(img_res, d=9, sigmaColor=75, sigmaSpace=75)
    templ = cv2.bilateralFilter(templ, d=9, sigmaColor=75, sigmaSpace=75)
    # experimental pts 
    experiment_points = []
    print(f"OD DO: {lower_scale}-{upper_scale}")
    for i in np.arange(lower_scale, upper_scale, 0.01):
        print(i)
        new_width = int(wt * i)
        new_height = int(ht * i)
        if new_height == 0:
            new_height = 1
        if new_width == 0:
            new_width = 1

        if percent > 2.0:
            templ = cv2.resize(templ, (new_width, new_height), interpolation=cv2.INTER_CUBIC)
        else:
            templ = cv2.resize(templ, (new_width, new_height), interpolation=cv2.INTER_AREA)
        if template_method == "TM_CCOEFF_NORMED":
            result_matrix = cv2.matchTemplate(img_res, templ, method=cv2.TM_CCOEFF_NORMED)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result_matrix)
            if 0.5 < max_val:
                #print(max_val)
                final_points.append([max_val, max_loc, i, 0])
        else:
            supl_img = cv2.resize(img_res,(templ.shape[1], templ.shape[0]),interpolation=cv2.INTER_AREA) # resize pomocnog imagea na vel templatea da bi se mogao podeliti
            zero_indices = np.where(templ == 0)
            if len(zero_indices[0]) > 0:
            # Postoji nula u denominatore, izbegavamo deljenje nulom
                print("Deljenje nulom detektovano. Postavljanje vrednosti na neku drugu vrednost.")
            # Primer postavljanja vrednosti na 1, možete koristiti bilo koju drugu vrednost
            templ[zero_indices] = 1
            templ = supl_img/templ 
            templ = cv2.convertScaleAbs(templ)
            result_matrix = cv2.matchTemplate(img_res, templ, method=cv2.TM_SQDIFF_NORMED)
            rslt = np.where(result_matrix < 0.2) #0.65 
            ml, _, mv, _ = cv2.minMaxLoc(result_matrix)
            print("MIN VAL - MIN LOC:", mv, ml, "SCALING: ",i)
            for pt in zip(*rslt[::-1]):#zip(rslt[1], rslt[0]):  # zip(*satisfied_points[::-1])
                if i>=0.29:
                    #print("TACKA (0,10) SCALE 29+%%:", result_matrix[10,0])
                    print(f"VALUE SQDIFF FOR {i} is on location {pt[1]}-{pt[0]} and is:", result_matrix[pt[1], pt[0]])
                final_points.append([pt, i, result_matrix[pt[1], pt[0]]])
            # crop = img_res[visina:visina+int(templ.shape[0]), sirina:sirina+int(templ.shape[1])]
            # cv2.imshow('crop', crop)
            # cv2.imshow('templ', templ)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
    return final_points


def getCDF(image):
    cdf, bins = cumulative_distribution(image)
    cdf = np.insert(cdf, 0, [0] * bins[0])
    cdf = np.append(cdf, [1] * (255 - bins[-1]))
    return cdf


def jensen_shannon_divergence(cdf1, cdf2):
    eps = 1e-10
    m = 0.5 * (cdf1 + cdf2)
    return 0.5 * (np.sum(cdf1 * np.log2((cdf1 + eps) / (m + eps))) + np.sum(cdf2 * np.log2((cdf2 + eps) / (m + eps))))


def chi_square_distance(cdf1, cdf2):
    return np.sum((cdf1 - cdf2) ** 2 / (cdf1 + cdf2 + 1e-10))


def ks_statistic(cdf1, cdf2):
    from scipy.stats import ks_2samp
    return ks_2samp(cdf1, cdf2)[0]


def correlation_coefficient(cdf1, cdf2):
    corr_coef = np.corrcoef(cdf1, cdf2)[0, 1]
    scaled_correlation = (corr_coef + 1) / 2
    return scaled_correlation


def erase_false_positives(image, template):
    from collections import defaultdict
    from operator import itemgetter
    cdf_template = getCDF(template)
    pixels_ndarray = image[0]
    top_left_coord = image[1]
    scaling_info = image[2]
    metric = image[3]
    pixels_ndarray = cv2.resize(pixels_ndarray, (template.shape[1], template.shape[0]), interpolation=cv2.INTER_AREA)
    cdf_image = getCDF(pixels_ndarray)
    correlation = correlation_coefficient(cdf_template, cdf_image)
    # cv2.imshow('templ',template)
    # cv2.imshow(f'img {metric}', pixels_ndarray)
    # cv2.waitKey(0)
    #cv2.destroyAllWindows()
    jsd = jensen_shannon_divergence(cdf_template, cdf_image)
    hsd = chi_square_distance(cdf_template, cdf_image)
    kss = ks_statistic(cdf_template, cdf_image)
    return [top_left_coord, correlation, jsd, hsd, scaling_info, kss, metric]


def mean_euclidean_dist(image, template):
    import numpy as np
    difference = 0
    for pix1, pix2 in zip(image, template):
        difference += np.sqrt(np.sum((pix1 - pix2) ** 2))
    return difference / (image.shape[0] * image.shape[1])


def is_grayscale_bgr(image, threshold=0.00):
    bgr_image = image
    gray_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2GRAY)
    mean_intensity_bgr = np.mean(bgr_image)
    mean_intensity_gray = np.mean(gray_image)
    if np.abs(mean_intensity_bgr - mean_intensity_gray) == threshold:
        return True  # Slika izgleda kao grayscale
    else:
        return False


final_points_locals = []
img, templ, dng_indicator = read_picture('slike2/folder10image8.png', 'template2/folder10template8.png') 
if is_grayscale_bgr(img) == True:
    is_grayscale = True
    template_method = "TM_SQDIFF_NORMED"
    rgb_diff = False
else:
    is_grayscale = False
    template_method = "TM_CCOEFF_NORMED"
    rgb_diff = True
print("DNG INDICATOR:", dng_indicator)
templ = template_crop(templ)
image_for_cdf = img
template_for_cdf = templ
percent, ht, wt = get_max_scaling(img, templ)
num_intervals = int(percent / 0.1)  # max broj intervala
intervals = split_interval(0.04, percent, num_intervals)
# res: lista svih najboljih lokalnih regija

#experimental_points = [] #-----------------------------------------------------

for interval in intervals:
    points = get_best_local_match(img, templ, interval[0], interval[1], percent, template_method)
    if (len(points) != 0):
        for point in points:
            final_points_locals.append(point)
        # for exp in exps:
        #     experimental_points.append(exp) # -----------------------------------------
    else:
        continue
print("final pts locals",len(final_points_locals))
if is_grayscale == True:
    exp_differences = []  #crp, templ  
    for expo in final_points_locals:  #   [(sirina,visina),i]      
        sir, vis = expo[0]
        scale_ex = expo[1]
        metric_val = expo[2]
        crp = image_for_cdf[vis:vis + int(scale_ex * template_for_cdf.shape[0]) + 1, sir:sir + int(scale_ex * template_for_cdf.shape[1]) + 1]
        tmpl = cv2.resize(template_for_cdf, (int(crp.shape[1]), int(crp.shape[0])), interpolation=cv2.INTER_AREA)     
        exp_difference = 0
        
        crp = cv2.bilateralFilter(crp,d=9,sigmaColor=75,sigmaSpace=75)
        tmpl = cv2.bilateralFilter(tmpl,d=9,sigmaSpace=75,sigmaColor=75)
        crp = cv2.cvtColor(crp, cv2.COLOR_BGR2GRAY)
        tmpl = cv2.cvtColor(tmpl, cv2.COLOR_BGR2GRAY)
        
        #tmpl = crp/tmpl # podelimo  ----------------------------------------------------
#     #print("SHAPES AFTER GRAY:", crp.shape,tmpl.shape)
        for pix1, pix2 in zip(crp, tmpl):
            if crp.shape != tmpl.shape:
                print("Pogrešan shape!")
            exp_difference += np.sqrt(np.sum((pix1 - pix2) ** 2))
#         #print("DIFFERENCE",exp_difference)
        exp_differences.append([crp,tmpl, exp_difference / (crp.shape[1] * crp.shape[0]),(sir,vis),scale_ex, metric_val])
# print(exp_differences)
    best_gs = sorted(exp_differences, key=lambda el:0.5*el[2]+0.5*el[5], reverse=False)[0:1] #bilo 2, False
    for expo_diff in best_gs:
        bw, bh = expo_diff[3]
        bs = expo_diff[4]
        cv2.rectangle(img, (bw, bh), (bw + int(wt * bs), bh + int(ht * bs)),
                      (0, 255, 0), 2)
        cv2.imshow(f"Result {bh}-{bw},scale: {bs}, metric: {expo_diff[2]}", img)
        cv2.imshow("Template", template_for_cdf)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
else:
    color_filtered_list = []
    if rgb_diff == False:
        color_filtered_list = final_points_locals # odmah prebaci fpl u colorfiltered list
    else:
        template_channels = cv2.mean(templ)
        template_channels = np.array([template_channels[0], template_channels[1], template_channels[2]])
        for point_info in final_points_locals:
            point = point_info[1]
            cropped_img = img[point[1]:point[1] + int(ht * point_info[2]), point[0]:point[0] + int(wt * point_info[2])]
            cropped_channels = cv2.mean(cropped_img)
            cropped_channels = np.array([cropped_channels[0], cropped_channels[1], cropped_channels[2]])
            diff_observation = cropped_channels - template_channels
            total_diff = np.sum(np.absolute(diff_observation))
            print(total_diff)
            if total_diff < 200:
                color_filtered_list.append([point_info[0], point_info[1], point_info[2]])

#2. erasing false positives with cdf correlation metric

    images = []
    upper_lefts = []
    print("COLOR FILTERED LIST:", len(color_filtered_list))
    hm_list = []
    for best_match in color_filtered_list:
        rec_w, rec_h = best_match[1]
        pixels = img[rec_h:rec_h + int(ht * best_match[2]), rec_w:rec_w + int(best_match[2] * wt)]
        template_cdf = cv2.resize(template_for_cdf, (int(best_match[2] * wt) + 1, int(best_match[2] * ht) + 1),
                              interpolation=cv2.INTER_AREA)
        image_for_cdf = [pixels, (rec_w, rec_h), best_match[2], best_match[0]]
        best_upper_left = erase_false_positives(image_for_cdf, template_cdf)
        upper_lefts.append(best_upper_left)

    print("ZADOVOLJILO RGB DIFF:", len(upper_lefts))

            #best_points = sorted(upper_lefts, key=lambda x:0.5*x[1]+0.5*x[6], reverse=True)[0:10]
    if dng_indicator == False:  # Treba False, necemo racunati Euklidovu distancu za dng format zato sto tu ne postoje false positives, tj. najbolji rezultat metrike korelacija*0.5 + tm_ccoeff_normed*0.5 vraca pravi template u originalu
        #3. euclidean distance for color  matching for top 10 cdf points
        best_points = sorted(upper_lefts, key=lambda x: 0.5 * x[1] + 0.5 * x[6], reverse=True)[0:10]
        euclideans = []
        print("BEST POINTS:", len(best_points))
        for point in best_points:
            width, height = point[0]
            scale = point[4]

            pixels_t = img[height:height + int(ht * scale), width:width + int(scale * wt)]
            template_t = cv2.resize(template_for_cdf, (pixels_t.shape[1], pixels_t.shape[0]),
                                    interpolation=cv2.INTER_AREA)
            mean_ed = mean_euclidean_dist(pixels_t, template_t)
            euclideans.append([point[0], point[1], point[2], point[3], point[4], point[5], point[6], mean_ed])

        for best_point in sorted(euclideans, key=lambda el: el[7], reverse=False)[0:2]:
            best_w, best_h = best_point[0]
            best_scaling = best_point[4]
            cv2.rectangle(img, (best_w, best_h), (best_w + int(wt * best_scaling), best_h + int(ht * best_scaling)),
                          (0, 255, 0), 2)
    else:  # necemo racunati prosecnu euklidovu distancu - ovo je dng format
        best_points = sorted(upper_lefts, key=lambda x: 0.5 * x[1] + 0.5 * x[6], reverse=True)[0:10]
        for best_point in best_points[0:1]:
            best_w, best_h = best_point[0]
            best_scaling = best_point[4]
            cv2.rectangle(img, (best_w, best_h), (best_w + int(wt * best_scaling), best_h + int(ht * best_scaling)),
                          (0, 255, 0), 2)

    cv2.imshow("Result", img)
    cv2.imshow("Template", template_for_cdf)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

DNG INDICATOR: False
Template sme da se povećava do 33.33333333333333% svoje originalne veličine.
OD DO: 0.04-0.13777777777777778
0.04
MIN VAL - MIN LOC: (0, 0) 1.0 SCALING:  0.04
0.05
Deljenje nulom detektovano. Postavljanje vrednosti na neku drugu vrednost.
MIN VAL - MIN LOC: (57, 26) 0.05074414610862732 SCALING:  0.05
0.060000000000000005
MIN VAL - MIN LOC: (0, 0) 1.0 SCALING:  0.060000000000000005
0.07
MIN VAL - MIN LOC: (41, 25) 0.040893442928791046 SCALING:  0.07
0.08000000000000002
MIN VAL - MIN LOC: (0, 0) 1.0 SCALING:  0.08000000000000002
0.09000000000000001
MIN VAL - MIN LOC: (46, 24) 0.06783439218997955 SCALING:  0.09000000000000001
0.1
MIN VAL - MIN LOC: (0, 0) 1.0 SCALING:  0.1
0.11000000000000001
MIN VAL - MIN LOC: (22, 24) 0.08224409073591232 SCALING:  0.11000000000000001
0.12000000000000002
MIN VAL - MIN LOC: (0, 0) 1.0 SCALING:  0.12000000000000002
0.13000000000000003
MIN VAL - MIN LOC: (0, 21) 0.08616919070482254 SCALING:  0.13000000000000003
OD DO: 0.1377777777777777

In [190]:
image = cv2.imread('slike2/folder6image7.png')
template = cv2.imread('template2/folder6template7.png')
image = cv2.resize(image,None, fx=1, fy=1)
# template = cv2.resize(template,None, fx=0.33, fy=0.33)
template = cv2.resize(template,(image.shape[1], image.shape[0]))
# 

# image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
# template = cv2.cvtColor(template, cv2.COLOR_BGR2YUV) 

res = image/(1*template) # deljenje i original je najjace 
#podeljeno_5 = image*0.2
print(cv2.mean(image))
mnoz = image*template 
delj_mean = image/73.31
oduzimanje = image-template 
sabiranje = image + template
cv2.imshow('delj mean', delj_mean)
#cv2.imshow('deljenje 5', podeljeno_5)
cv2.imshow('sabiranje',sabiranje)
cv2.imshow('mnozenje', mnoz)
cv2.imshow('deljenje',res)
cv2.imshow('oduzimanje', oduzimanje)
cv2.imshow('img', image)
cv2.imshow('templ', template)
cv2.waitKey(0)
cv2.destroyAllWindows()

(73.31521484375, 73.31521484375, 73.31521484375, 0.0)
